**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from sklearn import datasets
from sklearn.cluster import MiniBatchKMeans, DBSCAN, AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform
from collections import defaultdict
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML, SVG
from IPython.utils.capture import capture_output
from io import BytesIO

# get matplotblib's default color cycle
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

def plot_clust(
    X, clusts, is_core=None, iclust=None, legend=True, ax=None,
    core_point_size=100, other_point_size=50, rasterized=False
):
    if ax is None:
        ax = plt.gca()

    if is_core is None:
        is_core = np.full(len(X), False)

    if iclust is None:
        iclust = np.max(clusts)

    index = clusts >= 0
    regular = np.where(index)[0]
    out_of_cluster = np.where(~index)[0]

    c = [colors[i] for i in clusts[regular]]
    s = [core_point_size if is_core[i] else other_point_size for i in regular]

    ax.scatter(X[regular, 0], X[regular, 1], c=c, s=s, rasterized=rasterized)
    ax.scatter(X[out_of_cluster, 0], X[out_of_cluster, 1], c='black',
        s=other_point_size, rasterized=rasterized)

    if legend:
        legend_patches = [mpatches.Patch(color=colors[i], label='Cluster {}'.format(i)) for i in range(iclust+1)]
        if len(out_of_cluster):
            legend_patches.append(mpatches.Patch(color='black', label='Out of cluster'))
        ax.legend(handles=legend_patches)

    ax.grid(ls='--')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

class DBScanAnimation:
    def __init__(
        self, X, min_pts=5, eps=0.5,
        fig=None, ax=None,
        highlight_pt=True, pt_circle=True,
        plot_callback=plot_clust,
        core_point_size=100,
        other_point_size=25,
        keep_ax_lims=True,
        **kwargs
    ):
        self.X = X
        self.min_pts = min_pts
        self.eps = eps

        self.highlight_pt = highlight_pt
        self.pt_circle = pt_circle
        self.core_point_size = core_point_size
        self.other_point_size = other_point_size

        self.keep_ax_lims = keep_ax_lims
        self.plot_callback = plot_callback
        self.kwargs = kwargs

        with capture_output() as io:
            if fig is None:
                self.fig = plt.figure()
            else:
                self.fig = fig

            if ax is None:
                self.ax = self.fig.gca()
            else:
                self.ax = ax

            plt.show()

        self.iclust = None
        self.is_core = None
        self.clusts = None
        self.clust_candidates = None
        self.clust_set = None
        self.cpt = None
        self.D = None
        self.lims = None

    def plot_func(
        self, cpt=None, cpt_size=None, cpt_color=None,
        circle_color='red', ax=None
    ):
        if ax is None:
            ax = self.ax

        if cpt_size is None:
            cpt_size = self.core_point_size

        if cpt_color is None:
            cpt_color = 'red'

        ax.clear()
        
        self.plot_callback(
            X=self.X, clusts=self.clusts, is_core=self.is_core,
            iclust=self.iclust, ax=ax,
            core_point_size=self.core_point_size,
            other_point_size=self.other_point_size,
            **self.kwargs
        )
        
        if self.highlight_pt and not cpt is None:
            ax.scatter(self.X[cpt, 0], self.X[cpt, 1], c=cpt_color, s=cpt_size)

            if self.pt_circle:
                circle = plt.Circle(
                    (X[cpt, 0], X[cpt, 1]),
                    self.eps, color=circle_color, fill=False, linewidth=3
                )

                ax.add_patch(circle)

        if self.keep_ax_lims and not self.lims is None:
            ax.set_xlim(self.lims[0])
            ax.set_ylim(self.lims[1])

    def __call__(self):
        self.iclust = -1
        self.clusts = np.full(len(self.X), -1)
        self.is_core = np.full(len(self.X), False)
        self.clust_candidates = set(range(len(self.X)))
        self.D = squareform(pdist(X))
        self.lims = None

        self.plot_func()
        self.lims = self.ax.get_xlim(), self.ax.get_ylim()
        yield

        self.iclust = 0

        while len(self.clust_candidates):
            self.cpt = self.clust_candidates.pop()
            nbrs = np.where((self.D[self.cpt] < self.eps) & (self.clusts == -1))[0]

            if len(nbrs) < self.min_pts:
                continue
            
            self.is_core[self.cpt] = True
            self.clusts[self.cpt] = self.iclust

            self.clusts[nbrs] = self.iclust
            self.clust_candidates.difference_update(nbrs)
            self.clust_set = set(nbrs)

            self.plot_func(self.cpt)
            yield

            while len(self.clust_set):
                self.cpt = self.clust_set.pop()
                self.clusts[self.cpt] = self.iclust

                neighbours_index = self.D[self.cpt] < self.eps
                not_in_clust_index = self.clusts == -1
                in_same_clust_index = self.clusts == self.iclust

                if (neighbours_index & (not_in_clust_index | in_same_clust_index)).sum() < self.min_pts:
                    continue

                nbrs = np.where(neighbours_index & not_in_clust_index)[0]

                self.is_core[self.cpt] = True
                self.clusts[nbrs] = self.iclust
                self.clust_candidates.difference_update(nbrs)    
                self.clust_set.update(nbrs)

                if len(nbrs):
                    self.plot_func(self.cpt)
                    yield

            self.iclust += 1

        self.iclust -= 1
        self.plot_func()
        yield

        return self.clusts, self.is_core

    def save_images(self, image_filename_tpl="output/dbscan_{step}.svg"):
        for istep, _ in enumerate(self()):
            self.fig.savefig(
                image_filename_tpl.format(step=istep),
                bbox_inches='tight'
            )
            
    def grab_frame(self, format='svg'):
        file = BytesIO()
        dbscan_ani.fig.savefig(fname=file, format=format)
        file.seek(0)
        image_str = file.read()
        return image_str
            
    def make_all_frames(self, format='svg'):
        gen = self()
        return [self.grab_frame(format=format) for _ in gen]

    def make_animation(self, interval=500):
        ani = FuncAnimation(
            self.fig, lambda x: [], frames=self,
            interval=interval, repeat=False
        )
        
        return ani

    def animate(self, interval=500):
        ani = self.make_animation(interval=interval)
        return self.show_animation(ani)
    
    def show_animation(self, ani):
        html = HTML(ani.to_jshtml())
        return html
    
# algos and hyperparams for the comparison of methods
algos = [
    ('kmeans', MiniBatchKMeans),
    ('hierarchical, ward', AgglomerativeClustering),
    ('hierarchical, single', AgglomerativeClustering),
    ('DBSCAN', DBSCAN)
]

hyperparams = defaultdict(dict)
hyperparams['kmeans']['circles'] = {'n_clusters': 2}
hyperparams['kmeans']['moons'] = {'n_clusters': 2}
hyperparams['kmeans']['blobs'] = {'n_clusters': 3}
hyperparams['kmeans']['varied_blobs'] = {'n_clusters': 3}

hyperparams['hierarchical, ward']['circles'] = {'n_clusters': 2, 'linkage': 'ward'}
hyperparams['hierarchical, ward']['moons'] = {'n_clusters': 2, 'linkage': 'ward'}
hyperparams['hierarchical, ward']['blobs'] = {'n_clusters': 3, 'linkage': 'ward'}
hyperparams['hierarchical, ward']['varied_blobs'] = {'distance_threshold': 75, 'n_clusters': None, 'linkage': 'ward'}

hyperparams['hierarchical, single']['circles'] = {'n_clusters': 2, 'linkage': 'single'}
hyperparams['hierarchical, single']['moons'] = {'n_clusters': 2, 'linkage': 'single'}
hyperparams['hierarchical, single']['blobs'] = {'n_clusters': 3, 'linkage': 'single'}
hyperparams['hierarchical, single']['varied_blobs'] = {'distance_threshold': 1.55, 'n_clusters': None, 'linkage': 'single'}

hyperparams['DBSCAN']['circles'] = {'eps': 0.15, 'min_samples': 5}
hyperparams['DBSCAN']['moons'] = {'eps': 0.18, 'min_samples': 5}
hyperparams['DBSCAN']['blobs'] = {'eps': 1.5, 'min_samples': 5}
hyperparams['DBSCAN']['varied_blobs'] = {'eps': 5.0, 'min_samples': 5}

## DBSCAN: Zhlukovanie na báze hustoty

DBSCAN [[dbscan1996]](#dbscan1996) je známa metóda zhlukovania založená na hustote. Na rozdiel od metódy $k$-means či od hierarchického zhlukovania, počet zhlukov nie je potrebné špecifikovať vopred (explicitne ani implicitne špecifikovaním hladiny, kde rozrezať dendrogram). Hlavná myšlienka je takáto:

* Ak má bod aspoň $\textrm{min\_pts}$ susedov vzdialených $\leq \epsilon$, je to **jadrový bod**  (angl. core point) zhluku;


* Všetky ďalšie body vzdialené $\leq \epsilon$ sú s ním spojené a patria do toho istého zhluku;


* Ak sú tieto pripojené body:


* *Tiež jadrové body* , potom to isté platí aj o bodoch vzdialených $\leq \epsilon$ od nich, atď.;


* *Nie jadrové body* , potom sa nazývajú **hraničné body**  (border points) a nedajú sa cez ne do zhluku pripojiť už žiadne ďalšie body.



Pomocou týchto princípov je DBSCAN schopný objaviť zhluky. Keď sa bez DBSCAN-u skončí, niektoré body môžu zostať izolované, t.j. nie sú pripojené ku žiadnemu zhluku. Tieto sa jednoducho označia ako **mimozhlukové body**  – sú príliš ďaleko od ktoréhokoľvek zhluku na to, aby boli zaradené medzi jeho body.

### Demonštrácia

Teraz keď sme sa oboznámili so základnými princípmi, ukážeme si jednoduchú demonštráciu. Vytvoríme syntetickú dátovú množinu s troma zhlukmi guľového tvaru.



In [ ]:
X, Y = datasets.make_blobs(
    n_samples=75,
    centers=[(1, 3), (3, 5), (5, 2)],
    cluster_std=0.4,
    random_state=0
)

plt.scatter(X[:, 0], X[:, 1], c='black')
plt.grid(ls='--')
plt.xlabel('x')
plt.ylabel('y')

Následne si prejdeme krok za krokom, čo vykonáva DBSCAN (použijeme vlastnú triedu `DBScanAnimation`). Na začiatku nebudú existovať žiadne zhluky.



In [ ]:
dbscan_ani = DBScanAnimation(X=X, eps=0.5, min_pts=5)
frames = dbscan_ani.make_all_frames()
display(SVG(frames[0]))

V prvom kroku začneme potom náhodne (resp. v ľubovoľnom poradí – ak chceme, môžeme postupovať aj systematicky) vyberať body kým nenájdeme prvý jadrový bod (t.j. taký, ktorý má aspoň $\textrm{min\_pts}$ susedov vzdialených $\leq \epsilon$). Obrázok zvýrazňuje jeden taký bod červenou farbou spoločne s jeho $\epsilon$ okolím.

Z tohto bodu začneme vytvárať prvý zhluk. Zhluk bude na začiatku obsahovať jadrový bod a jeho $\epsilon$ okolie (body sú v grafe farebne kódované ako zhluk 0).



In [ ]:
display(SVG(frames[1]))

V ďalšom kroku prejdeme všetky tieto pridané body a určíme, ktoré z nich sú tiež jadrové (v grafe sú znázornené väčšími krúžkami) a následne pridáme do zhluku aj ich $\epsilon$ okolia atď. Body, ktoré patria do zhluku, ale nie sú jadrové, nazývame hraničnými bodmi. Tieto síce patria do zhluku, ale už sa cez ne nedajú pripojiť do zhluku ďalšie body.

Pozrime sa na niekoľko prvých krokov tohto procesu.



In [ ]:
display(SVG(frames[2]))
display(SVG(frames[3]))

display(HTML("<center><strong>⋮</strong></center>"))

display(SVG(frames[7]))

Potom, ako vyčerpáme všetky body pripojené k prvému zhluku, pokračujeme ďalej so zostávajúcimi mimozhlukovými bodmi. Snažíme sa medzi nimi nájsť ďalší jadrový bod, ktorý by sa dal použiť na vytvorenie nasledujúceho zhluku. Ďalej už postupujeme analogicky.



In [ ]:
display(SVG(frames[8]))

Napokon takto prejdeme všetky body. Môže sa stať, že aj potom zostanú niektoré body stále **mimozhlukové** . To je v poriadku – znamená to, že nespĺňajú kritériá na to, aby boli zaradené do jedného zo zhlukov. V praktických aplikáciách bude často užitočné, ak budeme vedieť také body identifikovať.



In [ ]:
display(SVG(frames[-1]))

Napokon si ešte skontrolujme jeden z hraničných bodov. Mali by sme vidieť, že hoci je v $\epsilon$ okolí jadrového bodu, nemá vo svojom vlastnom $\epsilon$ okolí dosť bodov na to, aby sa stal jadrovým bodom.



In [ ]:
ax = plt.gca()
boundary_point = np.where(dbscan_ani.clusts != -1 & ~dbscan_ani.is_core)[0][0]
dbscan_ani.plot_func(cpt=boundary_point, ax=ax, cpt_size=30)

Napokon môžeme vygenerovať ešte aj animovanú verziu celého procesu, aby ste si mohli prejsť podľa záujmu všetky kroky.



In [ ]:
dbscan_ani = DBScanAnimation(X=X, eps=0.5, min_pts=5)
dbscan_ani.animate()

### DBSCAN v balíčku scikit-learn

Teraz keď rozumieme, ako metóda DBSCAN funguje, pozrime sa, ako sa dá aplikovať v praxi pomocou balíčka `scikit-learn`. Nečakajú nás žiadne prekvapenia – DBSCAN používa štandardné `scikit-learn` rozhranie. "Číslo zhluku" pre mimozhlukové body je `-1`. Číslovanie bežných zhlukov začína od 0.

Upozorňujeme, že vo všeobecnosti je vhodné pred aplikáciou DBSCAN-u **dáta štandardizovať**  (presne ako sme to robili pri iných metódach zhlukovania). Tu štandardizáciu vynechávame jedine preto, že ide o čisto iustračné príklady a dáta si štandardizáciu reálne nevyžadujú.



In [ ]:
model = DBSCAN(eps=0.5, min_samples=5)
clusts = model.fit_predict(X)

In [ ]:
plot_clust(X=X, clusts=clusts)

### Metódy zhlukovania: porovnania

Napokon sa pozrieme ešte na malé porovnanie metód $k$-means, hierarchického zhlukovania (s ward a single linkage) a DBSCAN. Použijeme niekoľko pokusných dátových množín, aby sme lepšie ilustrovali, aké typy zhlukov je možné odhaliť pomocou každej z metód. Vygenerujme a vizualizujme si najprv samotné dáta: budeme používať dáta z [príkladu z dokumentácie ku scikit-learn-u](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html).



In [ ]:
#@title -- Data generation code -- { display-mode: "form" }
n_samples = 1500
circles, _ = datasets.make_circles(n_samples=n_samples, factor=0.5, noise=0.05, random_state=0)
moons, _ = datasets.make_moons(n_samples=n_samples, noise=0.05, random_state=0)
blobs, _ = datasets.make_blobs(n_samples=n_samples, random_state=8)
varied_blobs, _ = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=170)

data = [
    ('circles', circles),
    ('moons', moons),
    ('blobs', blobs),
    ('varied_blobs', varied_blobs)
]

# plotting
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))

axes[0, 0].scatter(circles[:, 0], circles[:, 1])
axes[0, 0].set_title('circles')
axes[0, 0].grid(ls='--')
axes[0, 0].set_ylabel('y')

axes[0, 1].scatter(moons[:, 0], moons[:, 1])
axes[0, 1].set_title('moons')
axes[0, 1].grid(ls='--')

axes[1, 0].scatter(blobs[:, 0], blobs[:, 1])
axes[1, 0].set_title('blobs')
axes[1, 0].grid(ls='--')
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('y')

axes[1, 1].scatter(varied_blobs[:, 0], varied_blobs[:, 1])
axes[1, 1].set_title('varied blobs')
axes[1, 1].grid(ls='--')
axes[1, 1].set_xlabel('x')

Teraz budeme na každú dátovú množinu aplikovať jednotlivé metódy zhlukovania (v každom prípade s trochu odlišnými hyperparametrami – tým sa však teraz nemusíme trápiť) a výsledky zobrazáme v grafe.



In [ ]:
#@title -- Clustering code -- { display-mode: "form" }
fig, axes = plt.subplots(len(data), len(algos))
fig.set_size_inches(10, 8)

for jax in range(axes.shape[1]):
    axes[0][jax].set_title(algos[jax][0])

for iax in range(axes.shape[0]):
    for jax in range(axes.shape[1]):
        ax = axes[iax][jax]
        X = data[iax][1]
        alg = algos[jax][1]
        params = hyperparams[algos[jax][0]][data[iax][0]]
        
        model = alg(**params)
        clusts = model.fit_predict(X)
        plot_clust(X=X, clusts=clusts, ax=ax, legend=False,
            other_point_size=5, rasterized=True)
        
        if jax > 0:
            ax.set_ylabel('')
            
        if iax < axes.shape[0] - 1:
            ax.set_xlabel('')

#### $k$-means

Vec, ktorú si všimneme okamžite je, že metóda $k$-means v skutočnosti funguje správne len so zhlukmi guľovitého tvaru – preto bude dobre fungovať len s dátovou množinou blobs.

#### Hierarchické zhlukovanie

Pri hierarchickom zhlukovaní môžeme meniť linkage metódy (ako sa určujú vzdialenosti medzi zhlukmi). Všimnite si, že ward linkage (a ďalšie ako sú average a complete linkage) nefunguje správne s množinami circles a moons. Single linkage metóda sa pozerá na najmenšie vzdialenosti medzi zhlukmi, takže je zameraná viac lokálne a funguje v týchto prípadoch lepšie. Nefunguje však zase veľmi dobre v prípade zhlukov s rozdielnymi hustotami.

#### DBSCAN

Podobne aj metóda DBSCAN funguje veľmi dobre s množinami blobs, circles a moons, ale – keďže je založená na hustote – **nedokáže sa vysporiadať so zhlukmi, ktoré majú podstatne odlišné hustoty – neexistuje žiadna vhodná hodnota pre $\epsilon$, pretože bude buď príliš nízka pre jeden zhluk alebo príliš vysoká pre iný.** 

Aby bolo zrejmejšie, v čom je problém, spustíme si ešte na množine s rozličnými hustotami zhlukov DBSCAN s dvoma rôznymi hodnotami $\epsilon$.



In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

model = DBSCAN(eps=5, min_samples=5)
clusts = model.fit_predict(varied_blobs)
plot_clust(X=varied_blobs, clusts=clusts, ax=axes[0],
    legend=False, other_point_size=10, rasterized=True)

model = DBSCAN(eps=0.5, min_samples=5)
clusts = model.fit_predict(varied_blobs)
plot_clust(X=varied_blobs, clusts=clusts, ax=axes[1],
    legend=False, other_point_size=10, rasterized=True)

### References

<a id="dbscan1996">[dbscan1996]</a> Ester, M., Kriegel, H.P., Sander, J. and Xu, X., 1996, August. A density-based algorithm for discovering clusters in large spatial databases with noise. In kdd (Vol. 96, No. 34, pp. 226-231).

